In [2]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import torchvision.transforms as transforms
import torch.nn.functional as F

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [33]:
import cv2
import numpy as np
from ultralytics import YOLO

# Initialize video capture
cap = cv2.VideoCapture(r'C:\Users\ahmad\Downloads\853889-hd_1920_1080_25fps.mp4')

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Load the YOLOv8 model and force CPU usage
model = YOLO("yolov8m.pt")  # Replace with the path to your YOLOv8 .pt file
model.to('cpu')  # Explicitly move the model to CPU

# Initialize tracker list (we'll use a list to keep track of individual trackers)
trackers = []  # List to store trackers for each object

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Check if the frame is empty or invalid
    if frame is None or frame.size == 0:
        print("Warning: Empty or invalid frame captured.")
        continue

    # Convert the frame to RGB (YOLOv8 expects RGB format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run YOLOv8 object detection
    results = model(frame_rgb)

    # Parse detection results
    detections = []  # List to store detected bounding boxes

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes in [x1, y1, x2, y2] format
        confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
        class_ids = result.boxes.cls.cpu().numpy()  # Class IDs

        # Iterate over detections and add them to the detection list
        for i, box in enumerate(boxes):
            confidence = confidences[i]
            if confidence > 0.5:  # Confidence threshold for detection
                # Get box coordinates
                x1, y1, x2, y2 = box
                detections.append([x1, y1, x2, y2, confidence])

    # Create new trackers for new detections
    for detection in detections:
        x1, y1, x2, y2, _ = detection
        tracker = cv2.TrackerKCF_create()  # You can change the tracker type (e.g., KCF, MIL, TLD, etc.)
        trackers.append(tracker)
        tracker.init(frame, (int(x1), int(y1), int(x2 - x1), int(y2 - y1)))  # Initialize tracker

    # Update trackers and draw bounding boxes with object IDs
    for i, tracker in enumerate(trackers):
        success, box = tracker.update(frame)
        if success:
            x1, y1, w, h = [int(v) for v in box]
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {i+1}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with bounding boxes and IDs
    cv2.imshow('Video', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 34 persons, 1 handbag, 367.1ms
Speed: 2.0ms preprocess, 367.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: module 'cv2' has no attribute 'TrackerKCF_create'